In [1]:
import os
import psycopg2
from sqlalchemy import create_engine
from dotenv import load_dotenv
import pandas as pd

In [2]:
# Parâmetros de conexão com o PostgreSQL
db_host = 'localhost'
db_port = os.getenv('DB_EXTERNAL_PORT')
db_database = os.getenv('DB_DATABASE')
db_user = os.getenv('DB_USER')
db_password = os.getenv('DB_PASSWORD')

In [3]:
try:
    # Testar conexão antes de prosseguir
    test_conn = psycopg2.connect(
        host=db_host,
        port=db_port,
        database=db_database,
        user=db_user,
        password=db_password
    )
    print("Conexão com o PostgreSQL estabelecida com sucesso!")
    test_conn.close()
except Exception as e:
    print(f"Erro ao estabelecer conexão com o PostgreSQL: {e}")

Conexão com o PostgreSQL estabelecida com sucesso!


In [6]:
engine = create_engine(f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_database}')

query = """-- Listar Tabelas
SELECT table_schema, table_name, 'table' AS type
FROM information_schema.tables
WHERE table_schema IN ('bronze', 'silver', 'gold')
  AND table_type = 'BASE TABLE'

UNION

-- Listar Views
SELECT table_schema, table_name, 'view' AS type
FROM information_schema.views
WHERE table_schema IN ('bronze', 'silver', 'gold')

UNION

-- Listar Views Materializadas
SELECT schemaname, matviewname AS table_name, 'materialized view' AS type
FROM pg_matviews
WHERE schemaname IN ('bronze', 'silver', 'gold')
ORDER BY table_schema, table_name;
"""
available_tables = pd.read_sql(query, con=engine)
available_tables

,table_schema,table_name,type
0,bronze,brt_data,table
1,gold,vw_brt_last_info,materialized view


In [7]:
pd.read_sql("SELECT * FROM gold.vw_brt_last_info", con=engine)

,codigo,data,hora,latitude,longitude,velocidade
0,0014003216,2025-01-23,02:14:30,-22.919666,-43.607588,0.0
1,00B8006F63 - A8 PRO,2025-01-17,02:38:56,-22.920816,-43.608415,0.0
2,00CF000457 - A16 MAX,2025-01-23,02:39:49,-22.916876,-43.606951,0.0
3,13409,2023-04-26,21:10:13,-22.915786,-43.628365,0.0
4,7000,2025-01-23,03:00:24,-22.874650,-43.226796,0.0
...,...,...,...,...,...,...
751,KXE9191 PKP,2025-01-23,02:33:34,-22.957008,-43.390301,0.0
752,KXM6403 SAVEIRO,2025-01-23,02:33:28,-23.000576,-43.363439,0.0
753,LSG7008 ACELLO INFRA,2025-01-18,17:03:03,-22.964554,-43.391436,0.0
754,RJG7D28 MOTO,2025-01-22,08:44:38,-23.000742,-43.364571,0.0
